In this notebook we propose a method to efficiently cluster time-space data using the existing DBSCAN implementation in [scikit-learn](http://scikit-learn.org/). Before taking this approach, we considered two other approaches to time-space clustering. The first was to use an [existing implementation](https://github.com/eubr-bigsea/py-st-dbscan) of [ST-DBSCAN](http://www.sciencedirect.com/science/article/pii/S0169023X06000218). This implementation uses [COMPSS](https://www.bsc.es/research-and-development/software-and-apps/software-list/comp-superscalar/) for distributed computing, which may be nontrivial to quickly deploy. The second approach that we considered was to implement a custom time-space metric in python which we could pass into the scikit-learn DBSCAN implementation. However, we were concerned that this approach would not scale very well to large data sets compared to metrics predefined (and optimized) in scikit-learn.

Our approach is to precompute a sparse spatial distance matrix where a given pairwise distance is only included if the time-distance is smaller than a given threshold. That is, for pairs of data points, we do the following:

1. If the time distance is smaller than a given threshold value, and
2. If the space distance is smaller than a second given threshold, then
3. Include the space distance in the sparse pairwise distance matrix

One way to think of this approach is that we are defining a time-space metric in which the time component only takes two values: smaller-than-epsilon and infinity.

If the input data set is sorted by timestamp, then computing this distance matrix can be very efficient because you only ever need to compare pairs of points that are physically near each other in the data. In fact, it may be possible to modify this approach to apply and record a custom time-space metric (i.e., to try the second approach that we rejected for scaling badly when directly supplied to the DBSCAN implementation).

Compared to ST-DBSCAN, this approach incorporates the time-distance in a very crude way and we expect the results will reflect that.

In [ ]:
# Import dependencies

import collections
import csv
import numpy as np
import pandas as pd
import os
import scipy as sp

from datetime import datetime, timedelta
from sklearn.cluster import DBSCAN
from sklearn.neighbors import DistanceMetric

In [ ]:
# Define the function used to compute the sparse distance matrix

"""Return the index of the first element x in a collection such that pred(x) is True."""
def find(coll, pred):
    for i, x in enumerate(coll):
        if pred(x):
            return i
    raise ValueError

"""Drop the first n elements from the left end of deq."""
def dropn(deq, n):
    for _ in range(n):
        deq.popleft()

"""
Create a sparse distance matrix of distances between points in the data set. Currently this 
function hardcodes certain things that we might want to make into parameters, such as the 
distance metric and the formatof the input CSV. In order to work correctly, this function 
requires that the input data is sorted byincreasing timestamp values.

infile - path to input CSV file
outfile - path where the output CSV file will be saved
time_threshold - only record distances where the time difference is smaller than this timedelta
space_threshold - only record distances if the space distance is smaller than this number
"""
def write_sparse_distance_matrix(infile, outfile, time_threshold, space_threshold):
    
    # We can make these parameters if we need to
    dist = DistanceMetric.get_metric("haversine")
    date_format = "%m/%d/%Y %H:%M"
    
    def go(rr, wr):
        left_index = 0
        right_index = 0
        
        time_window = collections.deque()  # timestamp
        space_window = collections.deque() # [lat, long]
        
        for row in rr:
            lat, long, current_ts = row['lat'], row['lon'], row['date'] 
            current_coords = [lat, long]
            current_ts = datetime.strptime(current_ts, date_format)
            
            try:
                number_to_drop = find(time_window, lambda ts: current_ts - ts <= time_threshold)
                left_index += number_to_drop
                dropn(time_window, number_to_drop)
                dropn(space_window, number_to_drop)
            except ValueError:
                left_index += len(time_window)
                time_window.clear()
                space_window.clear()
            
            if len(space_window) > 0:
                distances = dist.pairwise(space_window, [current_coords])
                for i, d in enumerate(np.nditer(distances)):
                    if d <= space_threshold:
                        wr.writerow({
                                "x": left_index + i,
                                "y": right_index,
                                "distance": d
                            })
                        wr.writerow({
                                "x": right_index,
                                "y": left_index + i,
                                "distance": d
                            })
            
            right_index += 1
            time_window.append(current_ts)
            space_window.append(current_coords)
        
    with open(infile) as f, open(outfile, 'w') as outf:
        rr = csv.DictReader(f)
        wr = csv.DictWriter(outf, fieldnames=["x", "y", "distance"])
        wr.writeheader()
        go(rr, wr)

In [ ]:
# Set the input and output locations

working_directory = "/path/to/data"
infile = os.path.join(working_directory, "summer-travel-gps-full.csv")
outfile = os.path.join(working_directory, "sparse_distance_matrix.csv")

# Set the threshold parameters for the distance calculation
# The space threshold should be the same value used as a parameter in DBSCAN below

time_threshold = timedelta(minutes=60)
space_threshold = 10

In [ ]:
# Write the sparse matrix into a CSV file

write_sparse_distance_matrix(infile, outfile, time_threshold, space_threshold)

In [ ]:
# Load the sparse distance matrix and compute clusters using DBSCAN

df = pd.read_csv(outfile)
distances = sp.sparse.csr_matrix((df["distance"], (df["x"], df["y"])))

model = DBSCAN(metric="precomputed", eps=space_threshold)
fit = model.fit(distances)